In [801]:
import pandas as pd
import json
import re

In [802]:
file_path = 'QA.thuvienphapluat.csv'
data = pd.read_csv(file_path)

In [803]:
# Drop rows missing values
data_cleaned = data.dropna()

In [804]:
cleaned_row_count = data_cleaned.shape[0]
print(f'Number of rows after cleaning: {cleaned_row_count}')

Number of rows after cleaning: 5753


In [805]:
# Function to transform the 'references' column into a list of references and replace old document names with new one
updated_name = "153/2020/NĐ-CP"


def transform_references(reference_str):
    references = reference_str.split(",")
    return [
        ref.strip().replace("65/2022/NĐ-CP", updated_name).replace("08/2023/NĐ-CP", updated_name) for ref in references
    ]


data_cleaned["references"] = data_cleaned["references"].apply(transform_references)
data_cleaned

/var/folders/q6/s8kvyvsx44x8dbnf9_kpzwgc0000gn/T/ipykernel_69762/3004869338.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned["references"] = data_cleaned["references"].apply(transform_references)


,question,answer,references
0,Hội đồng giải thể doanh nghiệp do Nhà nước nắm...,Theo quy định tại Điều 44 Nghị định 23/2022/NĐ...,[Điều 44 Nghị định 23/2022/NĐ-CP]
1,Có được quyền kéo dài thêm thời hạn giải thể d...,Theo quy định tại Điều 47 Nghị định 23/2022/NĐ...,[Điều 47 Nghị định 23/2022/NĐ-CP]
2,Khi giải thể doanh nghiệp do Nhà nước nắm giữ ...,Theo quy định tại khoản 2 Điều 39 Nghị định 23...,[khoản 2 Điều 39 Nghị định 23/2022/NĐ-CP]
3,Hồ sơ giải thể doanh nghiệp mới nhất năm 2023 ...,Căn cứ theo quy định tại khoản 1 Điều 210 Luật...,[khoản 1 Điều 210 Luật Doanh nghiệp 2020]
4,Thứ tự thanh toán khoản nợ của doanh nghiệp gi...,Theo quy định tại khoản 5 Điều 208 Luật Doanh ...,[khoản 5 Điều 208 Luật Doanh nghiệp 2020]
...,...,...,...
6684,Cá nhân sản xuất đồ gỗ xây dựng muốn đăng ký h...,Cá nhân sản xuất đồ gỗ xây dựng muốn đăng ký h...,[khoản 2 Điều 87 Nghị định 01/2021/NĐ-CP]
6686,Tài sản góp vốn khi thành lập doanh nghiệp có ...,Tài sản góp vốn khi thành lập doanh nghiệp đượ...,[Điều 34 Luật Doanh nghiệp 2020]
6687,Công ty TNHH 2 thành viên trở lên sẽ chia lợi ...,Nguyên tắc chia lợi nhuận cho các thành viên t...,[khoản 2 Điều 24 Luật Doanh nghiệp 2020]
6688,Điều kiện để công ty TNHH 2 thành viên trở lên...,Để chia lợi nhuận cho các thành viên thì công ...,[Điều 69 Luật Doanh nghiệp 2020]


In [806]:
# Function to clean the 'references' column, keeping only "Điều ... Luật Doanh nghiệp" or "Điều ... Nghị định"
def clean_references(reference_list):
    cleaned_references = []
    for ref in reference_list:
        match = re.search(r'(Điều \d+ Luật Doanh nghiệp \d+|Điều \d+ Nghị định \d+/\d+/NĐ-CP)', ref)
        if match:
            cleaned_references.append(match.group(0))
    return cleaned_references
data_cleaned['references'] = data_cleaned['references'].apply(clean_references)

/var/folders/q6/s8kvyvsx44x8dbnf9_kpzwgc0000gn/T/ipykernel_69762/2454947037.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['references'] = data_cleaned['references'].apply(clean_references)


In [807]:
# Remove rows where the 'references' list is empty after cleaning
data_cleaned = data_cleaned[data_cleaned['references'].map(len) > 0]

In [808]:
# Function to split each reference into ['Điều <số>', '<văn bản>']
def split_references_correctly(reference_list):
    split_refs = []
    for ref in reference_list:
        if isinstance(ref, str):
            match = re.search(r'(Điều \d+)', ref)
            if match:
                split_refs.append([match.group(0), ref.replace(match.group(0) + ' ', '')])
    return split_refs

data_cleaned['references'] = data_cleaned['references'].apply(split_references_correctly)

/var/folders/q6/s8kvyvsx44x8dbnf9_kpzwgc0000gn/T/ipykernel_69762/3511028552.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['references'] = data_cleaned['references'].apply(split_references_correctly)


In [809]:
data_cleaned[data_cleaned["question"] == "Công bố thông tin định kỳ của doanh nghiệp bao gồm những tài liệu nào?"]

,question,answer,references
302,Công bố thông tin định kỳ của doanh nghiệp bao...,Căn cứ quy định tại khoản 16 Điều 1 Nghị định ...,"[[Điều 1, Nghị định 153/2020/NĐ-CP]]"
1840,Công bố thông tin định kỳ của doanh nghiệp bao...,Căn cứ quy định tại khoản 16 Điều 1 Nghị định ...,"[[Điều 1, Nghị định 153/2020/NĐ-CP]]"
3211,Công bố thông tin định kỳ của doanh nghiệp bao...,Căn cứ quy định tại khoản 16 Điều 1 Nghị định ...,"[[Điều 1, Nghị định 153/2020/NĐ-CP]]"
4680,Công bố thông tin định kỳ của doanh nghiệp bao...,Căn cứ quy định tại khoản 16 Điều 1 Nghị định ...,"[[Điều 1, Nghị định 153/2020/NĐ-CP]]"


In [810]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5561 entries, 0 to 6689
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   question    5561 non-null   object
 1   answer      5561 non-null   object
 2   references  5561 non-null   object
dtypes: object(3)
memory usage: 173.8+ KB


In [811]:
# Convert each record to JSON format with pretty printing for better readability
json_records_cleaned = data_cleaned.to_dict(orient='records')

In [812]:
# Write JSON data to file
output_path_corrected = 'qaset.json'
with open(output_path_corrected, 'w', encoding='utf-8') as file:
    json.dump(json_records_cleaned, file, ensure_ascii=False, indent=4)

print(f'JSON data has been saved to {output_path_corrected}')

JSON data has been saved to qaset.json


In [813]:
data_cleaned

,question,answer,references
0,Hội đồng giải thể doanh nghiệp do Nhà nước nắm...,Theo quy định tại Điều 44 Nghị định 23/2022/NĐ...,"[[Điều 44, Nghị định 23/2022/NĐ-CP]]"
1,Có được quyền kéo dài thêm thời hạn giải thể d...,Theo quy định tại Điều 47 Nghị định 23/2022/NĐ...,"[[Điều 47, Nghị định 23/2022/NĐ-CP]]"
2,Khi giải thể doanh nghiệp do Nhà nước nắm giữ ...,Theo quy định tại khoản 2 Điều 39 Nghị định 23...,"[[Điều 39, Nghị định 23/2022/NĐ-CP]]"
3,Hồ sơ giải thể doanh nghiệp mới nhất năm 2023 ...,Căn cứ theo quy định tại khoản 1 Điều 210 Luật...,"[[Điều 210, Luật Doanh nghiệp 2020]]"
4,Thứ tự thanh toán khoản nợ của doanh nghiệp gi...,Theo quy định tại khoản 5 Điều 208 Luật Doanh ...,"[[Điều 208, Luật Doanh nghiệp 2020]]"
...,...,...,...
6684,Cá nhân sản xuất đồ gỗ xây dựng muốn đăng ký h...,Cá nhân sản xuất đồ gỗ xây dựng muốn đăng ký h...,"[[Điều 87, Nghị định 01/2021/NĐ-CP]]"
6686,Tài sản góp vốn khi thành lập doanh nghiệp có ...,Tài sản góp vốn khi thành lập doanh nghiệp đượ...,"[[Điều 34, Luật Doanh nghiệp 2020]]"
6687,Công ty TNHH 2 thành viên trở lên sẽ chia lợi ...,Nguyên tắc chia lợi nhuận cho các thành viên t...,"[[Điều 24, Luật Doanh nghiệp 2020]]"
6688,Điều kiện để công ty TNHH 2 thành viên trở lên...,Để chia lợi nhuận cho các thành viên thì công ...,"[[Điều 69, Luật Doanh nghiệp 2020]]"
